<a href="https://colab.research.google.com/github/chandanareddy-enugala/NLP-SLU/blob/main/SupervisedNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Method 1**

In [28]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
import random
from datetime import datetime

from sklearn.model_selection import train_test_split

In [29]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional 
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

In [30]:
def load_data_as_dataFrame(filePath):
  testfile = open(filePath, 'r')
  words = []
  tags = []
  for line in testfile:
      pieces = line.rstrip("\n").split("\t")
      words.append(pieces[0])
      tags.append(pieces[1])

  # --------------------------------------------------------------- Preparing DataFrame
  data = pd.DataFrame(columns=["words", "tags"])
  data['words'] = [str(word) for word in words]
  data['tags'] = [str(tag) for tag in tags]

  return data

In [31]:
FilePath = "/content/drive/MyDrive/VGG/train.tsv"
data = load_data_as_dataFrame(FilePath)
data.head()

,words,tags
0,konsiltan,k-on-s-i-l-t-an
1,depotwa,d-e-p-o-t-w-a
2,sosyopwofesyonèl,s-o-s-y-o-p-w-o-f-e-s-y-o-n-è-l
3,vejetal,v-e-j-e-t-a-l
4,repibliye,r-e-p-i-b-l-i-y-e


In [32]:
def tag_labels(tags_letter):
  tags_label = []
  for letter in tags_letter:
    if len(letter)==1:
      tags_label.append("B")
    elif len(letter)==2:
      tags_label.append("B")
      tags_label.append("I")
    elif len(letter)==3:
      tags_label.append("B")
      tags_label.append("I")
      tags_label.append("I")
  return tags_label

print(tag_labels(['k', 'on', 's', 'i', 'l', 't', 'an']))

['B', 'B', 'I', 'B', 'B', 'B', 'B', 'B', 'I']


In [33]:
FilePath = "/content/drive/MyDrive/VGG/train.tsv"
data = load_data_as_dataFrame(FilePath)

data['tags_letters'] = data['tags'].apply(lambda x: x.split("-"))
# data['tags_labels'] = data['tags_letters'].apply(lambda x: ["B" if len(l)==1 else "I" for l in x])
data['tags_labels'] = data['tags_letters'].apply(tag_labels)
data['tags_letters_length'] = data['tags_letters'].apply(lambda x: len(x))

In [34]:
data.head()

,words,tags,tags_letters,tags_labels,tags_letters_length
0,konsiltan,k-on-s-i-l-t-an,"[k, on, s, i, l, t, an]","[B, B, I, B, B, B, B, B, I]",7
1,depotwa,d-e-p-o-t-w-a,"[d, e, p, o, t, w, a]","[B, B, B, B, B, B, B]",7
2,sosyopwofesyonèl,s-o-s-y-o-p-w-o-f-e-s-y-o-n-è-l,"[s, o, s, y, o, p, w, o, f, e, s, y, o, n, è, l]","[B, B, B, B, B, B, B, B, B, B, B, B, B, B, B, B]",16
3,vejetal,v-e-j-e-t-a-l,"[v, e, j, e, t, a, l]","[B, B, B, B, B, B, B]",7
4,repibliye,r-e-p-i-b-l-i-y-e,"[r, e, p, i, b, l, i, y, e]","[B, B, B, B, B, B, B, B, B]",9


In [35]:
max_tags_letters_length = data['tags_letters_length'].max()
max_tags_letters_length

16

In [36]:
all_Unique_tags_labels = ["<PAD>", "B", "I"]

all_Unique_tags_labels_Length = len(all_Unique_tags_labels)
print("Length of Unique Labels : ", all_Unique_tags_labels_Length)
print("Unique Lables : ", all_Unique_tags_labels)

print("=============================================")

all_UniqueLetters = []
for lettersList in data['tags_letters']:
  all_UniqueLetters += [letter for letter in lettersList] 
all_UniqueLetters = list(set(all_UniqueLetters))
all_UniqueLetters = ["<PAD>"]+all_UniqueLetters

all_UniqueLetters_Length = len(all_UniqueLetters)
print("Length of Unique Letters : ", all_UniqueLetters_Length)
print("Unique Letters : ", all_UniqueLetters)

Length of Unique Labels :  3
Unique Lables :  ['<PAD>', 'B', 'I']
Length of Unique Letters :  60
Unique Letters :  ['<PAD>', 'z', 'n', 'V', 'on', 't', 'ch', 'è', 'e', 'I', 's', 'M', 'T', 'S', 'K', 'E', 'i', 'An', 'o', 'ou', 'Z', 'p', 'k', 'È', 'ui', 'a', 'G', 'w', 'D', 'd', 'On', 'L', 'B', 'F', 'en', 'f', 'ò', 'O', 'A', 'R', 'v', 'W', 'Ch', 'an', 'l', 'b', 'ng', 'Ò', 'y', 'j', 'à', 'J', 'N', 'oun', 'P', 'm', 'Y', 'En', 'r', 'g']


In [37]:
letters2index = {w:i for i, w in enumerate(all_UniqueLetters)}
labels2index = {w:i for i, w in enumerate(all_Unique_tags_labels)}

In [38]:
data["tags_letters_to_index"] = data["tags_letters"].apply(lambda x: [letters2index[letter] for letter in x])
data["tags_labels_to_index"] = data["tags_labels"].apply(lambda x: [labels2index[letter.upper()] for letter in x])

In [39]:
data.head(20)

,words,tags,tags_letters,tags_labels,tags_letters_length,tags_letters_to_index,tags_labels_to_index
0,konsiltan,k-on-s-i-l-t-an,"[k, on, s, i, l, t, an]","[B, B, I, B, B, B, B, B, I]",7,"[22, 4, 10, 16, 44, 5, 43]","[1, 1, 2, 1, 1, 1, 1, 1, 2]"
1,depotwa,d-e-p-o-t-w-a,"[d, e, p, o, t, w, a]","[B, B, B, B, B, B, B]",7,"[29, 8, 21, 18, 5, 27, 25]","[1, 1, 1, 1, 1, 1, 1]"
2,sosyopwofesyonèl,s-o-s-y-o-p-w-o-f-e-s-y-o-n-è-l,"[s, o, s, y, o, p, w, o, f, e, s, y, o, n, è, l]","[B, B, B, B, B, B, B, B, B, B, B, B, B, B, B, B]",16,"[10, 18, 10, 48, 18, 21, 27, 18, 35, 8, 10, 48...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,vejetal,v-e-j-e-t-a-l,"[v, e, j, e, t, a, l]","[B, B, B, B, B, B, B]",7,"[40, 8, 49, 8, 5, 25, 44]","[1, 1, 1, 1, 1, 1, 1]"
4,repibliye,r-e-p-i-b-l-i-y-e,"[r, e, p, i, b, l, i, y, e]","[B, B, B, B, B, B, B, B, B]",9,"[58, 8, 21, 16, 45, 44, 16, 48, 8]","[1, 1, 1, 1, 1, 1, 1, 1, 1]"
5,mason,m-a-s-on,"[m, a, s, on]","[B, B, B, B, I]",4,"[55, 25, 10, 4]","[1, 1, 1, 1, 2]"
6,akt,a-k-t,"[a, k, t]","[B, B, B]",3,"[25, 22, 5]","[1, 1, 1]"
7,ewopeyèn,e-w-o-p-e-y-è-n,"[e, w, o, p, e, y, è, n]","[B, B, B, B, B, B, B, B]",8,"[8, 27, 18, 21, 8, 48, 7, 2]","[1, 1, 1, 1, 1, 1, 1, 1]"
8,Azèbayidjan,A-z-è-b-a-y-i-d-j-an,"[A, z, è, b, a, y, i, d, j, an]","[B, B, B, B, B, B, B, B, B, B, I]",10,"[38, 1, 7, 45, 25, 48, 16, 29, 49, 43]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]"
9,teworis,t-e-w-o-r-i-s,"[t, e, w, o, r, i, s]","[B, B, B, B, B, B, B]",7,"[5, 8, 27, 18, 58, 16, 10]","[1, 1, 1, 1, 1, 1, 1]"


In [16]:
data['tags_letters_to_index'] += data['tags_letters_length'].apply(lambda x: [letters2index["<PAD>"] for i in range(max_tags_letters_length-x)])
data['tags_labels_to_index'] += data['tags_letters_length'].apply(lambda x: [labels2index["<PAD>"] for i in range(max_tags_letters_length-x)])

In [62]:
k = data['tags_letters'].apply(lambda x: max([len(i) for i in x]))
k[k==3].index

Int64Index([766, 3309, 7796, 8458, 8498, 8771, 9456, 11696], dtype='int64')

In [17]:
data.head()

,words,tags,tags_letters,tags_labels,tags_letters_length,tags_letters_to_index,tags_labels_to_index
0,konsiltan,k-on-s-i-l-t-an,"[k, on, s, i, l, t, an]","[B, B, I, B, B, B, B, B, I]",7,"[22, 4, 10, 16, 44, 5, 43, 0, 0, 0, 0, 0, 0, 0...","[1, 1, 2, 1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, ..."
1,depotwa,d-e-p-o-t-w-a,"[d, e, p, o, t, w, a]","[B, B, B, B, B, B, B]",7,"[29, 8, 21, 18, 5, 27, 25, 0, 0, 0, 0, 0, 0, 0...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,sosyopwofesyonèl,s-o-s-y-o-p-w-o-f-e-s-y-o-n-è-l,"[s, o, s, y, o, p, w, o, f, e, s, y, o, n, è, l]","[B, B, B, B, B, B, B, B, B, B, B, B, B, B, B, B]",16,"[10, 18, 10, 48, 18, 21, 27, 18, 35, 8, 10, 48...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,vejetal,v-e-j-e-t-a-l,"[v, e, j, e, t, a, l]","[B, B, B, B, B, B, B]",7,"[40, 8, 49, 8, 5, 25, 44, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,repibliye,r-e-p-i-b-l-i-y-e,"[r, e, p, i, b, l, i, y, e]","[B, B, B, B, B, B, B, B, B]",9,"[58, 8, 21, 16, 45, 44, 16, 48, 8, 0, 0, 0, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]"


In [40]:
all_UniqueLetters_Length, max_tags_letters_length

(60, 16)

In [44]:
X = pad_sequences(maxlen=max_tags_letters_length, sequences=data['tags_letters_to_index'])

Y = pad_sequences(maxlen=max_tags_letters_length, sequences=data['tags_labels_to_index'])
Y = to_categorical(Y, num_classes=3)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=89)

In [45]:
X.shape, Y.shape

((12812, 16), (12812, 16, 3))

In [46]:
model = Sequential()
model.add(Embedding(input_dim=all_UniqueLetters_Length, output_dim=50, input_length=max_tags_letters_length))
model.add(Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1)))
model.add(TimeDistributed(Dense(3, activation="softmax")))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 16, 50)            3000      
                                                                 
 bidirectional_2 (Bidirectio  (None, 16, 200)          120800    
 nal)                                                            
                                                                 
 time_distributed_2 (TimeDis  (None, 16, 3)            603       
 tributed)                                                       
                                                                 
Total params: 124,403
Trainable params: 124,403
Non-trainable params: 0
_________________________________________________________________
None


In [57]:


history = model.fit(X_train, y_train, batch_size=5, epochs=1, validation_split=0.2, verbose=1)



1640/1640 [==============================] - 47s 28ms/step - loss: 0.0051 - accuracy: 0.9986 - val_loss: 8.3325e-04 - val_accuracy: 0.9997


In [60]:


def evaluateModel(model, X, Y):
  Y_pred = model.predict(X)
  Y_pred = np.argmax(Y_pred, axis=-1)
  Y_actual = np.argmax(Y, axis=-1)
  return Y_actual, Y_pred
  
Y_actual, Y_pred = evaluateModel(model, X_test, y_test)



In [61]:
predicted = [i for labelList in Y_pred for i in labelList]
actual = [i for labelList in Y_actual for i in labelList]

cm = confusion_matrix(actual, predicted)

TN = cm[0,0]
FN = cm[1,0]
FP = cm[0,1]
TP = cm[1,1]

precision = (TP)/(TP+FP)
recall = (TP)/(TP+FN)
fscore = (2 * (precision * recall)) / (precision + recall)

print("Precision : ", precision)
print("Recall : ",  recall)
print("FScore : ", fscore)

Precision :  0.9999369522728706
Recall :  0.9998739124952717
FScore :  0.9999054313904738
